In [1]:
import os
import json
import pickle
import random
from collections import defaultdict, Counter

from indra.literature.adeft_tools import universal_extract_text
from indra.databases.hgnc_client import get_hgnc_name, get_hgnc_id

from adeft.discover import AdeftMiner
from adeft.gui import ground_with_gui
from adeft.modeling.label import AdeftLabeler
from adeft.modeling.classify import AdeftClassifier
from adeft.disambiguate import AdeftDisambiguator, load_disambiguator


from adeft_indra.ground.ground import AdeftGrounder
from adeft_indra.model_building.s3 import model_to_s3
from adeft_indra.model_building.escape import escape_filename
from adeft_indra.db.content import get_pmids_for_agent_text, get_pmids_for_entity, \
    get_plaintexts_for_pmids

In [2]:
adeft_grounder = AdeftGrounder()

In [3]:
shortforms = ['MIP']
model_name = ':'.join(sorted(escape_filename(shortform) for shortform in shortforms))
results_path = os.path.abspath(os.path.join('../..', 'results', model_name))

In [4]:
miners = dict()
all_texts = {}
for shortform in shortforms:
    pmids = get_pmids_for_agent_text(shortform)
    if len(pmids) > 10000:
        pmids = random.choices(pmids, k=10000)
    text_dict = get_plaintexts_for_pmids(pmids, contains=shortforms)
    text_dict = {pmid: text for pmid, text in text_dict.items() if len(text) > 5}
    miners[shortform] = AdeftMiner(shortform)
    miners[shortform].process_texts(text_dict.values())
    all_texts.update(text_dict)

longform_dict = {}
for shortform in shortforms:
    longforms = miners[shortform].get_longforms()
    longforms = [(longform, count, score) for longform, count, score in longforms
                 if count*score > 2]
    longform_dict[shortform] = longforms
    
combined_longforms = Counter()
for longform_rows in longform_dict.values():
    combined_longforms.update({longform: count for longform, count, score
                               in longform_rows})
grounding_map = {}
names = {}
for longform in combined_longforms:
    groundings = adeft_grounder.ground(longform)
    if groundings:
        grounding = groundings[0]['grounding']
        grounding_map[longform] = grounding
        names[grounding] = groundings[0]['name']
longforms, counts = zip(*combined_longforms.most_common())
pos_labels = []

In [5]:
list(zip(longforms, counts))

[('molecularly imprinted polymer', 113),
 ('macrophage inflammatory protein', 69),
 ('maximal inspiratory pressure', 46),
 ('mycobacterium indicus pranii', 30),
 ('maximum intensity projection', 25),
 ('major intrinsic protein', 25),
 ('molecularly inversion probe', 14),
 ('maximum inspiratory pressure', 14),
 ('minimally invasive parathyroidectomy', 13),
 ('malaria in pregnancy', 12),
 ('myoinhibitory peptide', 11),
 ('myd88 inhibitory peptide', 8),
 ('mitochondrial intermediate peptidase', 6),
 ('main intrinsic protein', 6),
 ('mercury intrusion porosimetry', 4),
 ('macrophage infectivity potential', 4),
 ('micropapillary', 4),
 ('myo inositol 1 phosphate', 4),
 ('main intrinsic polypeptide', 3),
 ('methylation induced premeiotically', 3),
 ('mouse insulin promoter', 3),
 ('minimally invasive potential', 3),
 ('maximal inspiratory', 3),
 ('maff interacting protein', 3),
 ('inspiratory muscle strength', 3)]

In [21]:
grounding_map, names, pos_labels = ground_with_gui(longforms, counts, 
                                                   grounding_map=grounding_map,
                                                   names=names, pos_labels=pos_labels, no_browser=True, port=8891)

In [22]:
result = [grounding_map, names, pos_labels]

In [24]:
result

[{'inspiratory muscle strength': 'ungrounded',
  'macrophage infectivity potential': 'ungrounded',
  'macrophage inflammatory protein': 'FPLX:Macrophage_inflammatory_proteins',
  'maff interacting protein': 'HGNC:31102',
  'main intrinsic polypeptide': 'ungrounded',
  'main intrinsic protein': 'FPLX:AQP',
  'major intrinsic protein': 'FPLX:AQP',
  'malaria in pregnancy': 'ungrounded',
  'maximal inspiratory': 'NCIT:C135496',
  'maximal inspiratory pressure': 'NCIT:C135496',
  'maximum inspiratory pressure': 'NCIT:C135496',
  'maximum intensity projection': 'NCIT:C94930',
  'mercury intrusion porosimetry': 'ungrounded',
  'methylation induced premeiotically': 'ungrounded',
  'micropapillary': 'ungrounded',
  'minimally invasive parathyroidectomy': 'ungrounded',
  'minimally invasive potential': 'ungrounded',
  'mitochondrial intermediate peptidase': 'HGNC:7104',
  'molecularly imprinted polymer': 'molecularly imprinted polymer',
  'molecularly inversion probe': 'ungrounded',
  'mouse in

In [25]:
grounding_map, names, pos_labels = [{'inspiratory muscle strength': 'ungrounded',
  'macrophage infectivity potential': 'ungrounded',
  'macrophage inflammatory protein': 'FPLX:Macrophage_inflammatory_proteins',
  'maff interacting protein': 'HGNC:31102',
  'main intrinsic polypeptide': 'ungrounded',
  'main intrinsic protein': 'FPLX:AQP',
  'major intrinsic protein': 'FPLX:AQP',
  'malaria in pregnancy': 'ungrounded',
  'maximal inspiratory': 'NCIT:C135496',
  'maximal inspiratory pressure': 'NCIT:C135496',
  'maximum inspiratory pressure': 'NCIT:C135496',
  'maximum intensity projection': 'NCIT:C94930',
  'mercury intrusion porosimetry': 'ungrounded',
  'methylation induced premeiotically': 'ungrounded',
  'micropapillary': 'ungrounded',
  'minimally invasive parathyroidectomy': 'ungrounded',
  'minimally invasive potential': 'ungrounded',
  'mitochondrial intermediate peptidase': 'HGNC:7104',
  'molecularly imprinted polymer': 'molecularly imprinted polymer',
  'molecularly inversion probe': 'ungrounded',
  'mouse insulin promoter': 'ungrounded',
  'mycobacterium indicus pranii': 'TAXONOMY:35617',
  'myd88 inhibitory peptide': 'myd88_inhibitory_peptide',
  'myo inositol 1 phosphate': 'ungrounded',
  'myoinhibitory peptide': 'MESH:C000603492'},
 {'FPLX:Macrophage_inflammatory_proteins': 'Macrophage_inflammatory_proteins',
  'HGNC:31102': 'MAFIP',
  'FPLX:AQP': 'AQP',
  'NCIT:C135496': 'Maximal Inspiratory Pressure',
  'NCIT:C94930': 'Maximum Intensity Projection',
  'HGNC:7104': 'MIPEP',
  'molecularly imprinted polymer': 'molecularly imprinted polymer',
  'TAXONOMY:35617': 'Mycobacterium intracellulare subsp. intracellulare',
  'myd88_inhibitory_peptide': 'myd88 inhibitory peptide',
  'MESH:C000603492': 'myoinhibitory peptide, Drosophila'},
 ['FPLX:AQP',
  'FPLX:Macrophage_inflammatory_proteins',
  'HGNC:31102',
  'HGNC:7104',
  'MESH:C000603492']]

In [26]:
excluded_longforms = []

In [27]:
grounding_dict = {shortform: {longform: grounding_map[longform] 
                              for longform, _, _ in longforms if longform in grounding_map
                              and longform not in excluded_longforms}
                  for shortform, longforms in longform_dict.items()}
result = [grounding_dict, names, pos_labels]

if not os.path.exists(results_path):
    os.mkdir(results_path)
with open(os.path.join(results_path, f'{model_name}_preliminary_grounding_info.json'), 'w') as f:
    json.dump(result, f)

In [28]:
additional_entities = {}

In [29]:
unambiguous_agent_texts = {}

In [30]:
labeler = AdeftLabeler(grounding_dict)
corpus = labeler.build_from_texts((text, pmid) for pmid, text in all_texts.items())
agent_text_pmid_map = defaultdict(list)
for text, label, id_ in corpus:
    agent_text_pmid_map[label].append(id_)

entity_pmid_map = {entity: set(get_pmids_for_entity(*entity.split(':', maxsplit=1),
                                                major_topic=True))for entity in additional_entities}

In [12]:
intersection1 = []
for entity1, pmids1 in entity_pmid_map.items():
    for entity2, pmids2 in entity_pmid_map.items():
        intersection1.append((entity1, entity2, len(pmids1 & pmids2)))

In [13]:
intersection2 = []
for entity1, pmids1 in agent_text_pmid_map.items():
    for entity2, pmids2 in entity_pmid_map.items():
        intersection2.append((entity1, entity2, len(set(pmids1) & pmids2)))

In [14]:
intersection1

[('HGNC:1848', 'HGNC:1848', 86)]

In [15]:
intersection2

[('MESH:D018893', 'HGNC:1848', 0),
 ('HGNC:1848', 'HGNC:1848', 1),
 ('ungrounded', 'HGNC:1848', 0),
 ('CHEBI:CHEBI:64198', 'HGNC:1848', 0),
 ('MESH:D019164', 'HGNC:1848', 0),
 ('MESH:C059416', 'HGNC:1848', 0),
 ('CHEBI:CHEBI:15710', 'HGNC:1848', 0)]

In [17]:
all_used_pmids = set()
for entity, agent_texts in unambiguous_agent_texts.items():
    used_pmids = set()
    for agent_text in agent_texts[1]:
        pmids = set(get_pmids_for_agent_text(agent_text))
        new_pmids = list(pmids - all_texts.keys() - used_pmids)
        text_dict = get_plaintexts_for_pmids(new_pmids, contains=agent_texts)
        corpus.extend([(text, entity, pmid) for pmid, text in text_dict.items()])
        used_pmids.update(new_pmids)
    all_used_pmids.update(used_pmids)
        
for entity, pmids in entity_pmid_map.items():
    new_pmids = list(set(pmids) - all_texts.keys() - all_used_pmids)
    if len(new_pmids) > 10000:
        new_pmids = random.choices(new_pmids, k=10000)
    _, contains = additional_entities[entity]
    text_dict = get_plaintexts_for_pmids(new_pmids, contains=contains)
    corpus.extend([(text, entity, pmid) for pmid, text in text_dict.items()])

In [18]:
names.update({key: value[0] for key, value in additional_entities.items()})
names.update({key: value[0] for key, value in unambiguous_agent_texts.items()})
pos_labels = list(set(pos_labels) | additional_entities.keys() |
                  unambiguous_agent_texts.keys())

In [31]:
%%capture

classifier = AdeftClassifier(shortforms, pos_labels=pos_labels, random_state=1729)
param_grid = {'C': [100.0], 'max_features': [10000]}
texts, labels, pmids = zip(*corpus)
classifier.cv(texts, labels, param_grid, cv=5, n_jobs=5)

INFO: [2020-10-28 04:17:22] /adeft/PP/adeft/adeft/modeling/classify.py - Beginning grid search in parameter space:
{'C': [100.0], 'max_features': [10000]}
INFO: [2020-10-28 04:17:27] /adeft/PP/adeft/adeft/modeling/classify.py - Best f1 score of 0.8238476971669672 found for parameter values:
{'logit__C': 100.0, 'tfidf__max_features': 10000}


In [32]:
classifier.stats

{'label_distribution': {'HGNC:7104': 6,
  'FPLX:Macrophage_inflammatory_proteins': 63,
  'FPLX:AQP': 23,
  'ungrounded': 47,
  'NCIT:C135496': 42,
  'NCIT:C94930': 22,
  'molecularly imprinted polymer': 103,
  'HGNC:31102': 3,
  'TAXONOMY:35617': 17,
  'myd88_inhibitory_peptide': 5,
  'MESH:C000603492': 7},
 'f1': {'mean': 0.823848, 'std': 0.039743},
 'precision': {'mean': 0.812943, 'std': 0.052718},
 'recall': {'mean': 0.852101, 'std': 0.033838},
 'ungrounded': {'f1': {'mean': 0.689351, 'std': 0.129239},
  'pr': {'mean': 0.671111, 'std': 0.203743},
  'rc': {'mean': 0.740303, 'std': 0.064137}},
 'HGNC:7104': {'f1': {'mean': 0.933333, 'std': 0.133333},
  'pr': {'mean': 0.9, 'std': 0.2},
  'rc': {'mean': 1.0, 'std': 0.0}},
 'NCIT:C94930': {'f1': {'mean': 0.897691, 'std': 0.057377},
  'pr': {'mean': 0.95, 'std': 0.1},
  'rc': {'mean': 0.869524, 'std': 0.113393}},
 'MESH:C000603492': {'f1': {'mean': 0.733333, 'std': 0.38873},
  'pr': {'mean': 0.7, 'std': 0.4},
  'rc': {'mean': 0.8, 'std': 

In [33]:
disamb = AdeftDisambiguator(classifier, grounding_dict, names)

In [34]:
disamb.dump(model_name, results_path)

In [35]:
print(disamb.info())

Disambiguation model for MIP

Produces the disambiguations:
	AQP*	FPLX:AQP
	MAFIP*	HGNC:31102
	MIPEP*	HGNC:7104
	Macrophage_inflammatory_proteins*	FPLX:Macrophage_inflammatory_proteins
	Maximal Inspiratory Pressure	NCIT:C135496
	Maximum Intensity Projection	NCIT:C94930
	Mycobacterium intracellulare subsp. intracellulare	TAXONOMY:35617
	molecularly imprinted polymer	molecularly imprinted polymer
	myd88 inhibitory peptide	myd88_inhibitory_peptide
	myoinhibitory peptide, Drosophila*	MESH:C000603492

Class level metrics:
--------------------
Grounding                                         	Count	F1     
                     molecularly imprinted polymer	103	0.97714
                  Macrophage_inflammatory_proteins*	 63	0.86808
                                        Ungrounded	 47	0.68935
                      Maximal Inspiratory Pressure	 42	0.94314
                                               AQP*	 23	0.81801
                      Maximum Intensity Projection	 22	0.89769
Mycobacteri

In [36]:
model_to_s3(disamb)